In [119]:
import sqlite3
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import re

In [120]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("formula1.sqlite")
cur = con.cursor()

# Podiums over the years

In [116]:
query = """
With R as (SELECT raceId, year, name from races),
Re as (SELECT resultId, raceId, driverId, grid, position from results),
D as (Select driverId, surname from drivers)
Select R.raceid, D.driverid, surname, grid, position, year from Re
INNER JOIN R ON R.raceid == Re.raceid
INNER JOIN D ON D.driverid == Re.driverid
where position <= 3.0
order by year;
"""

In [53]:
df = pd.read_sql_query(query, con)
df

,raceId,driverId,surname,grid,position,year
0,833,642,Farina,1,1,1950
1,833,786,Fagioli,2,2,1950
2,833,686,Parnell,4,3,1950
3,834,579,Fangio,1,1,1950
4,834,647,Ascari,7,2,1950
...,...,...,...,...,...,...
3070,1029,842,Gasly,6,2,2019
3071,1029,832,Sainz,20,3,2019
3072,1030,1,Hamilton,1,1,2019
3073,1030,830,Verstappen,2,2,2019


# Races

In [72]:
query = """
With R as (SELECT raceId, circuitId, year, date from races),
C as (SELECT circuitid, name, location, country, lat, lng from circuits)
Select * from R
INNER JOIN C ON R.circuitid = C.circuitid
order by year;
"""

In [73]:
df = pd.read_sql_query(query, con)
df

,raceId,circuitId,year,date,circuitid,name,location,country,lat,lng
0,833,9,1950,1950-05-13,9,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694
1,834,6,1950,1950-05-21,6,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056
2,835,19,1950,1950-05-30,19,Indianapolis Motor Speedway,Indianapolis,USA,39.7950,-86.23470
3,836,66,1950,1950-06-04,66,Circuit Bremgarten,Bern,Switzerland,46.9589,7.40194
4,837,13,1950,1950-06-18,13,Circuit de Spa-Francorchamps,Spa,Belgium,50.4372,5.97139
...,...,...,...,...,...,...,...,...,...,...
1035,1048,22,2020,2020-10-11,22,Suzuka Circuit,Suzuka,Japan,34.8431,136.54100
1036,1049,69,2020,2020-10-25,69,Circuit of the Americas,Austin,USA,30.1328,-97.64110
1037,1050,32,2020,2020-11-01,32,Autódromo Hermanos Rodríguez,Mexico City,Mexico,19.4042,-99.09070
1038,1051,18,2020,2020-11-15,18,Autódromo José Carlos Pace,São Paulo,Brazil,-23.7036,-46.69970


# Circuits

In [123]:
query = """
Select name, url from circuits;
"""

In [124]:
df = pd.read_sql_query(query, con)
df

,name,url
0,Albert Park Grand Prix Circuit,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,Sepang International Circuit,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,Bahrain International Circuit,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,Circuit de Barcelona-Catalunya,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,Istanbul Park,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...
69,Red Bull Ring,http://en.wikipedia.org/wiki/Red_Bull_Ring
70,Sochi Autodrom,http://en.wikipedia.org/wiki/Sochi_Autodrom
71,Port Imperial Street Circuit,http://en.wikipedia.org/wiki/Port_Imperial_Str...
72,Baku City Circuit,http://en.wikipedia.org/wiki/Baku_City_Circuit
